In [60]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [61]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'tube_outside_diameter_sae'

In [62]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [63]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
field='TubeO_D_'
fields = [field]
# df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

3905


In [73]:
# start=r'(?i)(^.{0,10})|()'
# df['start']=df['product_name'].apply(lambda x: re_extract(start, str(x)))

pattern=r'''(?i)(.{0,20}(?:tubing|tube|\bOD\b|outside.?dia).{0,20})|()''' 
pattern=r'''(?i)(tube)|(\bod\b)|()'''
df['name'] = df['product_name'].apply(lambda x: re_extract(pattern, str(x)))
df['ld'] = df['long_desc'].apply(lambda x: re_extract(pattern, str(x)))



pat=r'(?i)(\d+.?\d*\/?\d*.{0,3}\d+.?\d*\/?\d*)|()'
df['matches']=df[field].apply(lambda x: re_extract(pat, str(x)))

pat=r'(?i)(m)|()'
df['mm']=df[field].apply(lambda x: re_extract(pat, str(x)))


custom=df[(df[field].astype(str)!='nan')&((df['mm'].astype(str)=='[]'))]
# custom[curation_col]=custom['matches'].apply(lambda x: re.sub(r'.+to\s?|','',str(x))).apply(lambda x: re.sub(r"'|\[|\]",'',str(x)))#.apply(lambda x: re.sub(r'(?i)\s?mm\s?',' mm',str(x))).apply(lambda x: re.sub(r'\s?"\s?',' in',str(x))).apply(lambda x: f'{x} in').apply(lambda x: re.sub(r'\s?in\s?in',' in',str(x)))                                          
custom[curation_col]=custom[field].apply(lambda x: f'{x}"').apply(lambda x: re.sub(r'""','"',str(x)))
print(len(custom))
matchcustom=custom[['external_id',curation_col]]
# custom[curation_col].explode().value_counts()

2626


In [79]:
not_match=df[(df[field].astype(str)=='nan')]
print(len(not_match[(not_match['ld'].astype(str)!='[]')|(not_match['name'].astype(str)!='[]')]))
print(len(not_match[(not_match['ld'].astype(str)=='[]')&(not_match['name'].astype(str)=='[]')]))
# not_match[(not_match['ld'].astype(str)!='[]')|(not_match['name'].astype(str)!='[]')][0:500]

544
340


In [75]:
# df[(df['name'].astype(str)!='[]')&(df['matches'].astype(str)!='[]')][0:500]

In [76]:
# two=df[(df['matches'].astype(str)=='[]')]
# print(len(two[two['name'].astype(str)!='[]']))
# two=two[two['name'].astype(str)!='[]']
# two[curation_col]=two['name'].apply(lambda x:re.sub(r'(?i)P\.?S\.?I\.?','',str(x))).apply(lambda x: re.sub(r'.+\-\s?',"['",str(x))).apply(lambda x: re.sub(r'(?<!\,)000',',000',str(x))).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"\s?'\]",'"]',str(x))).apply(lambda x: re.sub(r"\s?\s?\s?'\s?\,\s?'",'","',str(x))).apply(lambda x: re.sub(r'200"\,"125','125","200',str(x))).apply(lambda x: re.sub(r'\["1\,500"\,"100"\,"2,000 "\]','["100","1,500","2,000"]',str(x))).apply(lambda x: re.sub(r'\s?"\]','"]',str(x)))                                                    
# two[curation_col].explode().value_counts()
# matchtwo=two[['external_id',curation_col]]

In [77]:
# two=df[(df['matches'].astype(str)=='[]')]
# # two['name']=two['name'].apply(lambda x: re.sub(r'(?i)\(|x','',str(x))).apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'\s?\,\s?',',',str(x))).apply(lambda x: re.sub(r'"\]','""]',str(x))).apply(lambda x:re.sub(r'\[""|\["','["',str(x)))                                                                  
# pat=r'(?i)((?i).{0,20}dia.{0,20})|()'
# two['name']=two['product_name'].apply(lambda x: re_extract(pat, str(x)))
# two['long']=two['long_desc'].apply(lambda x: re_extract(pat, str(x)))
# two[(two['name'].astype(str)!='[]')|(two['long'].astype(str)!='[]')]

# N/A

In [78]:
na=df[(df['name'].astype(str)=='[]')&(df['ld'].astype(str)=='[]')]
print(len(na))
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
# print(len(na)+len(matchcustom)+len(matchtwo))

1360


In [34]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcustom))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchtwo))

# def get_df_name(df):
#     name =[x for x in gglobals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchfinal))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchex))

matchcustom
matchna


In [11]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [35]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchcustom) 

In [80]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 

In [47]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchtwo) 